#### Level-4 data of L1000 gene-expression profiling is calculated as:

Level 4 - Differential Expression (ZSPC):

To obtain a measure of relative gene expression, we use a robust z-scoring procedure to generate differential expression values from normalized profiles (Level-3). We compute the **differential expression of gene x in the ith sample(xi) on the plate (zi)** as
                              
                              zi = xi - median(X) / 1.4826 * MAD(X)


***where X is the vector of normalized gene expression of gene x across all samples on the plate, MAD is the median absolute deviation of X, and the factor of 1.4826 makes the denominator a consistent estimator of scale for normally distributed data.***

[Subramanian et al 2017](https://www.cell.com/action/showPdf?pii=S0092-8674%2817%2931309-0)



- Level-4 data - are replicate level data i.e. where you have multiple profiles been perturbed by the same compound (pertubagen)

#### The goal here:


-- is to determine the median score of each compound per dose based on taking the median of the correlation values between replicates of the same compound.

In [1]:
import os
import requests
import pickle
import argparse
import pandas as pd
import numpy as np
import re
from os import walk
from collections import Counter
import random
import shutil
from statistics import median
import cmapPy.pandasGEXpress.parse_gct as pg
from cmapPy.pandasGEXpress.parse import parse
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [2]:
data_dir = os.getcwd() ##current dir
zipurl = "https://ndownloader.figshare.com/articles/13181966/versions/1"
def download_L1000_data(data_dir, zipurl):
    """
    Download L1000 data from figshare and extract 
    the zipped files into a directory
    """
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
        
    with urlopen(zipurl) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(data_dir)

In [ ]:
download_L1000_data(data_dir, zipurl)

In [3]:
os.listdir(data_dir) ##files in L1000 downloaded dataset

['col_meta_level_3_n421176.txt',
 'col_meta_level_3_REP.A_A549_only_n27837.txt',
 'col_meta_level_5_n169494.txt',
 'col_meta_level_5_REP.A_A549_only_n9482.txt',
 'level_3_q2norm_n27837x978.gctx',
 'level_4W_zspc_n27837x978.gctx',
 'level_4_zspc_n27837x978.gctx',
 'level_5_modz_cid_n9482.grp',
 'level_5_modz_common_sigs_n8370x978.gctx',
 'level_5_modz_n9482x978.gctx',
 'level_5_rank_n9482x978.gctx',
 'REP.A_A549_pert_info.txt',
 'set_size_3_level_3_REP.A_A549_only_all_compounds_and_random_well_DMSO.gmt']

In [4]:
pertinfo_file = '../aligned_moa_CP_L1000.csv'

Two level-4 L1000 Data:

- 'level_4W_zspc_n27837x978.gctx'
- 'level_4_zspc_n27837x978.gctx',

In [5]:
def construct_lvl4_data(data_dir, pertinfo_file):
    """
    This function returns L1000 Level-4 Data that is aligned with 
    the important metadata information to compute compound's median scores
    """
    
    lvl4_data = parse(os.path.join(data_dir, 'level_4_zspc_n27837x978.gctx'))
    lvl4_data = lvl4_data.data_df.rename_axis(None).T
    lvl4_data = lvl4_data.rename_axis(None).reset_index()
    lvl4_data.rename(columns={"index": "replicate_id"}, inplace = True)
    df_metalvl_5 = pd.read_csv(os.path.join(data_dir, 'col_meta_level_5_REP.A_A549_only_n9482.txt'), delimiter = "\t")
    lvl4_data['sig_id'] = lvl4_data['replicate_id'].apply(lambda x: ''.join(re.sub('(24H.+(\_|\.)[A-Z0-9]+.)\:', '24H:', x)))
    df_meta_features = df_metalvl_5[['sig_id', 'pert_id', 'pert_idose']].copy()
    df_meta_features['dose'] = df_meta_features['pert_idose'].map({'-666' : 0, '0.04 uM' : 1, '0.12 uM' : 2, '0.37 uM' : 3,
                                                                   '1.11 uM' : 4, '3.33 uM' : 5, '10 uM' : 6, '20 uM' : 7})
    df_pertinfo = pd.read_csv(pertinfo_file)
    df_pertinfo.rename(columns={"broad_id": "pert_id"}, inplace = True)
    df_meta_features = pd.merge(df_meta_features, df_pertinfo, on=['pert_id'], how = 'left')
    lvl4_data = pd.merge(lvl4_data, df_meta_features, on='sig_id')
    
    return lvl4_data

In [6]:
df_level4 = construct_lvl4_data(data_dir, pertinfo_file)

In [7]:
df_level4.head()

,replicate_id,200814_at,222103_at,201453_x_at,204131_s_at,200059_s_at,205067_at,213702_x_at,214435_x_at,201334_s_at,...,211071_s_at,203341_at,205379_at,sig_id,pert_id,pert_idose,dose,Metadata_broad_sample,pert_iname,moa
0,REP.A001_A549_24H_X1_B27:A03,0.3547,-0.4940,-0.1721,-0.0339,-0.4355,1.8263,-0.1316,0.0853,-0.4660,...,-1.3157,1.0145,0.1046,REP.A001_A549_24H:A03,DMSO,-666,0,DMSO,DMSO,Control vehicle
1,REP.A001_A549_24H_X2_B27:A03,-0.0447,-1.6390,-0.5276,-0.5092,-0.5733,0.2445,0.6159,-1.1273,2.8250,...,0.4649,-0.3140,-1.5836,REP.A001_A549_24H:A03,DMSO,-666,0,DMSO,DMSO,Control vehicle
2,REP.A001_A549_24H_X3_B27:A03,-0.9583,0.2657,-0.8204,-0.5851,-2.0808,0.0739,-0.0497,-0.3012,-0.9787,...,-4.7854,-1.1885,-0.1416,REP.A001_A549_24H:A03,DMSO,-666,0,DMSO,DMSO,Control vehicle
3,REP.A001_A549_24H_X1_B27:A04,-0.2130,0.4931,-0.8768,-0.6968,-1.7018,-0.3779,-0.6745,-1.9799,-1.1429,...,-1.4900,0.9750,-1.8243,REP.A001_A549_24H:A04,DMSO,-666,0,DMSO,DMSO,Control vehicle
4,REP.A001_A549_24H_X2_B27:A04,0.7499,-1.2819,0.4981,1.7090,1.6765,-1.2690,1.7974,-1.2213,3.4625,...,1.0424,-0.7604,0.3157,REP.A001_A549_24H:A04,DMSO,-666,0,DMSO,DMSO,Control vehicle


### - Remove highly correlated landmark genes and samples with Null compound values

In [8]:
def feature_selection(df_data):
    
    """
    Perform feature selection by dropping columns with null compounds values, 
    and highly correlated landmark genes from the data.
    """
    
    df_data_genes = df_data.drop(['replicate_id', 'Metadata_broad_sample', 'pert_id', 'dose', 'pert_idose', 
                                  'pert_iname', 'moa', 'sig_id'], axis = 1).copy()
    df_data_corr = df_data_genes.corr(method = 'spearman')
    drop_cols = []
    n_cols = len(df_data_corr.columns)
    for i in range(n_cols):
        for k in range(i+1, n_cols):
            val = df_data_corr.iloc[k, i]
            col = df_data_corr.columns[i]
            if abs(val) >= 0.8:
                drop_cols.append(col)
    df_data.drop(set(drop_cols), axis = 1, inplace = True)
    df_data.drop(df_data[df_data['pert_iname'].isnull()].index).reset_index(drop = True, inplace = True)
    
    return df_data

In [9]:
df_level4 = feature_selection(df_level4)

In [10]:
df_level4.shape

(27837, 984)

In [11]:
def get_median_score(cpds_list, df):
    """
    This function calculates the median score for each compound based on its replicates
    """
    
    cpds_median_score = {}
    for cpd in cpds_list:
        cpd_replicates = df[df['pert_iname'] == cpd].copy()
        cpd_replicates.drop(['replicate_id', 'Metadata_broad_sample', 'pert_id', 'dose', 'pert_idose', 
                             'pert_iname', 'moa', 'sig_id'], axis = 1, inplace = True)
        cpd_replicates_corr = cpd_replicates.astype('float64').T.corr(method = 'spearman').values
        if len(cpd_replicates_corr) == 1:
            median_val = 1
        else:
            median_val = median(list(cpd_replicates_corr[np.triu_indices(len(cpd_replicates_corr), k = 1)]))
        
        cpds_median_score[cpd] = median_val
        
    return cpds_median_score

In [12]:
def check_compounds(cpd_med_score, df):
    """
    Check if all distinct compounds in the Level-4 dataframe are present 
    in the cpd_med_score dictionary, if not add the compounds as keys to the dictionary 
    and give them a null value.
    """
    cpd_list = df['pert_iname'].unique().tolist()
    cpd_keys = cpd_med_score.keys()
    for cpd in cpd_list:
        if cpd not in cpd_keys:
            cpd_med_score[cpd] = np.nan
            
    return cpd_med_score

In [13]:
def get_cpd_medianscores(df):
    
    """This function computes median scores for all compounds found in the Level-4 dataframe PER DOSE (1-6)"""
    
    dose_list = list(set(df['dose'].unique().tolist()))[1:7]
    
    for dose in dose_list:
        df_dose = df[df['dose'] == dose].copy()
        cpds_list = df_dose['pert_iname'].unique().tolist()
        cpds_median_score = get_median_score(cpds_list, df_dose)
        cpds_median_score = check_compounds(cpds_median_score, df)
        sorted_med_score = {key:value for key, value in sorted(cpds_median_score.items(), key=lambda item: item[0])}
        if dose == 1:
            df_cpd_med_score = pd.DataFrame.from_dict(sorted_med_score, orient='index', columns = ['dose_1'])
        else:
            df_cpd_med_score['dose_' + str(dose)] = sorted_med_score.values()
            
    return df_cpd_med_score

In [14]:
df_cpd_med_score = get_cpd_medianscores(df_level4)

In [15]:
df_cpd_med_score.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6
"16,16-dimethylprostaglandin-e2",0.175716,0.081803,0.296784,0.082931,NaN,NaN
17-hydroxyprogesterone-caproate,0.024949,0.036924,0.066243,0.145056,0.098257,0.180456
2-iminobiotin,-0.129397,0.234609,-0.042823,-0.028020,0.098841,0.098864
"3,3'-diindolylmethane",0.038456,0.012045,-0.009883,-0.014877,NaN,NaN
3-amino-benzamide,0.051625,-0.016660,0.004272,-0.009650,-0.011359,0.113275
3-deazaadenosine,0.064700,-0.058230,0.046483,-0.010995,-0.024314,0.090172
ABT-737,0.017857,0.017354,0.035991,0.021638,0.077127,0.154783
AICA-ribonucleotide,-0.007261,0.091987,0.033744,0.102993,0.011979,0.055921
AKT-inhibitor-1-2,-0.006916,0.138613,0.089607,0.076500,-0.031787,0.041715
ALX-5407,0.072244,0.064040,0.094309,0.007991,0.000050,0.072674


In [16]:
def drop_cpds_with_null(df):
    """
    This function drop compounds with median scores of 1 
    or null values in any of the dose points (1-6)
    """
    cpds_with_null = []
    for cpd in df.index:
        if any(df.loc[cpd] == 1) | any(df.loc[cpd].isnull()):
            cpds_with_null.append(cpd)
    df.drop(cpds_with_null, axis = 0, inplace = True)
    
    return df

In [17]:
df_cpd_med_score = drop_cpds_with_null(df_cpd_med_score)

In [18]:
df_cpd_med_score.shape

(1330, 6)

In [19]:
df_cpd_med_score.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6
17-hydroxyprogesterone-caproate,0.024949,0.036924,0.066243,0.145056,0.098257,0.180456
2-iminobiotin,-0.129397,0.234609,-0.042823,-0.028020,0.098841,0.098864
3-amino-benzamide,0.051625,-0.016660,0.004272,-0.009650,-0.011359,0.113275
3-deazaadenosine,0.064700,-0.058230,0.046483,-0.010995,-0.024314,0.090172
ABT-737,0.017857,0.017354,0.035991,0.021638,0.077127,0.154783
AICA-ribonucleotide,-0.007261,0.091987,0.033744,0.102993,0.011979,0.055921
AKT-inhibitor-1-2,-0.006916,0.138613,0.089607,0.076500,-0.031787,0.041715
ALX-5407,0.072244,0.064040,0.094309,0.007991,0.000050,0.072674
AS-605240,-0.204854,0.073841,-0.176075,-0.184031,0.176874,0.096799
AT-7519,0.125742,-0.008306,0.077190,0.441905,0.822501,0.850574


In [20]:
def no_of_replicates_per_cpd(df, df_lvl4):
    """This function computes the numbers of replicates for each compound (cpd_size)"""
    
    dose_list = list(set(df_lvl4['dose'].unique().tolist()))[1:7]
    cpds_size = {}
    for cpd in df.index:
        num_of_replicates = 0
        for dose in dose_list:
            df_dose = df_lvl4[df_lvl4['dose'] == dose].copy()
            cpd_replicates = df_dose[df_dose['pert_iname'] == cpd].copy()
            num_of_replicates += cpd_replicates.shape[0]
        cpd_replicate_length = num_of_replicates // len(dose_list)
        cpds_size[cpd] = cpd_replicate_length
    df['cpd_size'] = cpds_size.values()
    
    return df

In [21]:
df_cpd_med_score = no_of_replicates_per_cpd(df_cpd_med_score, df_level4)

In [22]:
df_cpd_med_score.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6,cpd_size
17-hydroxyprogesterone-caproate,0.024949,0.036924,0.066243,0.145056,0.098257,0.180456,3
2-iminobiotin,-0.129397,0.234609,-0.042823,-0.028020,0.098841,0.098864,2
3-amino-benzamide,0.051625,-0.016660,0.004272,-0.009650,-0.011359,0.113275,3
3-deazaadenosine,0.064700,-0.058230,0.046483,-0.010995,-0.024314,0.090172,2
ABT-737,0.017857,0.017354,0.035991,0.021638,0.077127,0.154783,3
AICA-ribonucleotide,-0.007261,0.091987,0.033744,0.102993,0.011979,0.055921,3
AKT-inhibitor-1-2,-0.006916,0.138613,0.089607,0.076500,-0.031787,0.041715,3
ALX-5407,0.072244,0.064040,0.094309,0.007991,0.000050,0.072674,3
AS-605240,-0.204854,0.073841,-0.176075,-0.184031,0.176874,0.096799,2
AT-7519,0.125742,-0.008306,0.077190,0.441905,0.822501,0.850574,3


In [23]:
def save_to_csv(df, path, file_name, compress=None):
    """saves dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index=False, compression=compress)

In [24]:
save_to_csv(df_cpd_med_score.reset_index().rename({'index':'cpd'}, axis = 1), 
            'L1000_lvl4_cpd_replicate_datasets', 'cpd_replicate_median_scores.csv')

In [25]:
save_to_csv(df_level4, 'L1000_lvl4_cpd_replicate_datasets', 
            'L1000_level4_cpd_replicates.csv.gz', compress="gzip")